# Importações

In [125]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, classification_report

import warnings
warnings.filterwarnings('ignore')

# 1. Data Loading

In [104]:
df = pd.read_csv('/home/esdras-daniel/Documentos/Python/PGM-DataAnalysis/data/processed/pgm-dataset-processado.csv')
df.head(3)

,teorTexto,setorDestino,tipoAviso,orgaoJulgador,assuntos,documentos,anexos,classeProcesso
0,PODER JUDICIÁRIO ESTADO DO RIO GRANDE DO NORTE...,Saúde,Intimação,2ª VIJ da Com. Natal,12485;12494,Decisão;Petição de manifestação;Despacho;Certi...,notaTecnica-228951.pdf - notaTecnica-228951;E-...,1706
1,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Administrativa,Intimação,3º JFP da Com. Natal,10715,Notificação;Petição;Despacho;Diligência;Notifi...,FICHA FINANCEIRA;SEMAD - Joelma;08464942120228...,14695
2,PODER JUDICIÁRIO DO ESTADO DO RIO GRANDE DO NO...,Fiscal,Intimação,5ª VEFT de Natal,9518,Despacho;Petição;Despacho;Petição;Despacho;Pet...,docs habilitação;Sisbajud Positivo;CÁLCULOS;Ex...,156


In [105]:
df['setorDestino'].value_counts()

setorDestino
Fiscal            6686
Administrativa    5403
Contabilidade     1338
Judicial          1137
Saúde              443
Meio Ambiente      155
Patrimonial        121
Name: count, dtype: int64

Um problema que podemos observar nos nossos dados é o desbalanceamento entre classes. Para um problema de classificação isso pode resultar em um classificador "preguiçoso" onde prefere apenas classificar as classes predominantes.

Porém, apenas fazer upsampling e downsampling pode trazer problemas de *leakage*, onde dados que não deveriam ser vistos pelo modelo são utilizados acidentalmente para o treinamento.

A melhor abordagem é primeiro separa os dados de treinamento e teste de forma **estratificada** e só aplicar o upsampling nos dados de treinamento.

<h3>Passos da Solução</h3>

1. Dividir os dados entre treinamento e teste de forma estratificada
    * Isso garante que o conjunto de teste mantenha a proporção original das classes.

2. Aplicar o *upsampling* apenas no conjunto de treinamento
    * Pode ser feito duplicando amostras da classe minoritária ou usando técnicas como SMOTE (que cria amostras sintéticas).

3. Treinar o modelo na base balanceada e calidar no conjunto de teste original
    * Isso garante que a avaliação reflete o desempenho real do modelo.    

## 1.1 - Separação dos dados de forma estratificada

In [106]:
from sklearn.model_selection import train_test_split

y = df['setorDestino'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.3, random_state=42, stratify=y)
print(f'Shape dos dados de treinamento: {X_train.shape}')
print(f'Shape dos dados de teste: {X_test.shape}')

Shape dos dados de treinamento: (10698, 8)
Shape dos dados de teste: (4585, 8)


## 1.2 - *Upsampling* e *Downsampling* dos dados de treinamento.

In [107]:
def balance_df(df: pd.DataFrame, balance_on:str = 'setorDestino', n_samples:int = 500):
    unique_classes = df[balance_on].unique()
    
    dfs_list = []

    for classe in unique_classes:
        if len(df[df[balance_on] == classe]) >= n_samples:
            dfs_list.append(df[df[balance_on] == classe].sample(n=n_samples, replace=False))
        else:
            dfs_list.append(df[df[balance_on] == classe].sample(n=n_samples, replace=True))
    
    df_balanceado = pd.concat(dfs_list)

    return df_balanceado, df_balanceado['setorDestino'].to_numpy()

X_train, y_train = balance_df(X_train, balance_on='setorDestino', n_samples=500)

In [108]:
print(f'Shape dos dados de treinamento: {X_train.shape} | {y_train.shape}')

print(f'\n\nDISTRIBUIÇÃO DOS DADOS DE TREINAMENTO\n{X_train['setorDestino'].value_counts()}')
print(f'\n\nDISTRIBUIÇÃO DOS DADOS DE TESTE\n{X_test['setorDestino'].value_counts()}')



Shape dos dados de treinamento: (3500, 8) | (3500,)


DISTRIBUIÇÃO DOS DADOS DE TREINAMENTO
setorDestino
Administrativa    500
Fiscal            500
Judicial          500
Patrimonial       500
Contabilidade     500
Meio Ambiente     500
Saúde             500
Name: count, dtype: int64


DISTRIBUIÇÃO DOS DADOS DE TESTE
setorDestino
Fiscal            2006
Administrativa    1621
Contabilidade      401
Judicial           341
Saúde              133
Meio Ambiente       47
Patrimonial         36
Name: count, dtype: int64


# 2. Construção da Classe Classificador Final.

![Classificador Final](../img/Classificador_Ensemble.drawio.png)

## 2.1 - Criação dos Transformadores

In [109]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import MultiLabelBinarizer
from typing import Tuple

# Transformador para a pipeline do sklearn

class ColumnExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, col_name: str):
        self.col_name = col_name
        # Inicializa One-Hot Encoder
        self.one_hot_encoding = None

    def fit(self, X: pd.DataFrame, y=None):
        """Ajusta os tranformadores nos dados de treinamento"""
        if not isinstance(X, pd.DataFrame):
            raise TypeError(f'Esperado um DataFrame, mas recebeu {type(X).__name__}.')
        
        # Verifica se a coluna existe no DataFrame
        if self.col_name not in X.columns:
            raise ValueError(f'A coluna {self.col_name} não foi encontrada no Dataframe.')
            
        # Ajusta o MultilabelBinarizer para as colunas categóricas
        self.one_hot_encoding = MultiLabelBinarizer()
        elements_list = X[self.col_name].apply(lambda x: x.split(';') if isinstance(x, str) else [x])
        self.one_hot_encoding.fit(elements_list)

        return self
    
    def transform(self, X: pd.DataFrame):
        """Transforma os dados usando modelos treinados"""
        if not isinstance(X, pd.DataFrame):
            raise TypeError(f'Esperado um DataFrame, mas recebeu {type(X).__name__}.')
        
        X_encoded = []

        # Faz o one-hot-encoding
        if not self.one_hot_encoding:
            raise ValueError(f'O MultiLabelBinarizer para "{self.col_name}" não foi treinado. Chame "fit" antes de "transform".')
            
        elements_list = X[self.col_name].apply(lambda x: x.split(';') if isinstance(x, str) else [x])

        return self.one_hot_encoding.transform(elements_list)

# 3. Pipeline Final

Definição da Pipeline 'final' (falta o classificador de regras), onde temos, um classificador AdaBoost junto com um classificador Stacking

In [110]:
'''from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier, StackingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression

ada_boost_pipeline = Pipeline([
                    ('feature_extraction', ColumnExtractor(('assuntos', 'classeProcesso', 'orgaoJulgador'))),
                    ('AdaBoost Classifier', AdaBoostClassifier(
                        estimator=LinearSVC(),
                        n_estimators=10,
                    ))
                ])

stacking_estimators = [
    ('SVM', Pipeline([
        ('feature_extraction', ColumnExtractor(('teorTexto', 'assuntos', 'classeProcesso'))), # , 'assuntos'
        ('classificator', LinearSVC())
    ])),
    ('RF', Pipeline([
        ('feature_extraction', ColumnExtractor('teorTexto')), # , 'assuntos', 'classeProcesso', 'orgaoJulgador'
        ('classficator', RandomForestClassifier())
    ])),
    ('Logistic Regression', Pipeline([
        ('feature_extraction', ColumnExtractor(('assuntos', 'classeProcesso', 'orgaoJulgador'))),
        ('classificator', LogisticRegression())
    ]))
]

stacking_classificator = StackingClassifier(
    estimators=stacking_estimators,
    final_estimator=LogisticRegression(),
    cv = 10
)'''

In [111]:
stacking_classificator

StackingClassifier(cv=10,
                   estimators=[('SVM',
                                Pipeline(steps=[('feature_extraction',
                                                 ColumnExtractor(col_names=('teorTexto',
                                                                            'assuntos',
                                                                            'classeProcesso'))),
                                                ('classificator',
                                                 LinearSVC())])),
                               ('RF',
                                Pipeline(steps=[('feature_extraction',
                                                 ColumnExtractor(col_names=('teorTexto',))),
                                                ('classficator',
                                                 RandomForestClassifier())])),
                               ('Logistic Regression',
                                Pipeline(steps=[('feature_extraction',
                                                 ColumnExtractor(col_names=('assuntos',
                                                                            'classeProcesso',
                                                                            'orgaoJulgador'))),
                                                ('classificator',
                                                 LogisticRegression())]))],
                   final_estimator=LogisticRegression())

In [126]:
stacking_classificator.fit(X_train, y_train)
y_pred = stacking_classificator.predict(X_test)

print(f'Acurácia: {accuracy_score(y_test, y_pred)*100:.5f}%')
print(f'Recall: {recall_score(y_test, y_pred, average='weighted')*100:.5f}%')
print(f'{classification_report(y_test, y_pred)}')
print(f'{confusion_matrix(y_test, y_pred)}')

Acurácia: 93.08615%
Recall: 93.08615%
                precision    recall  f1-score   support

Administrativa       0.96      0.92      0.94      1621
 Contabilidade       0.93      0.98      0.95       401
        Fiscal       0.99      0.97      0.98      2006
      Judicial       0.63      0.82      0.71       341
 Meio Ambiente       0.51      0.38      0.44        47
   Patrimonial       0.59      0.47      0.52        36
         Saúde       0.94      0.91      0.92       133

      accuracy                           0.93      4585
     macro avg       0.79      0.78      0.78      4585
  weighted avg       0.94      0.93      0.93      4585

[[1486   21   14   94    4    1    1]
 [   5  393    2    1    0    0    0]
 [  21    5 1953   21    4    1    1]
 [  35    2    1  280    8    9    6]
 [   2    1    3   22   18    1    0]
 [   3    1    0   15    0   17    0]
 [   1    0    0   10    1    0  121]]


# Apenas Teste de PipeLine

In [130]:
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder



# Vetorização do texto
text_transformer = Pipeline([
    ("tfidf", TfidfVectorizer(max_features=500))
])

# Codificação One-Hot para os outros atributos categóricos
categorical_transformer = Pipeline([
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

# Transformação das features
preprocessor = ColumnTransformer(transformers=[
    ("text", text_transformer, "teorTexto"),
    ("cat", categorical_transformer, ["assuntos", "classeProcesso", "orgaoJulgador"])
])

# Classificadores individuais
clf1 = RandomForestClassifier()
clf2 = LogisticRegression()
clf3 = SVC(probability=True)

# Pipeline para cada classificador
classifier1 = Pipeline([
    ("preprocessor", ColumnTransformer([("text", text_transformer, "teorTexto")])),
    ("clf", clf1)
])

classifier2 = Pipeline([
    ("preprocessor", ColumnTransformer([("cat", categorical_transformer, ["assuntos", "classeProcesso", "orgaoJulgador"])])),
    ("clf", clf2)
])

classifier3 = Pipeline([
    ("preprocessor", preprocessor),
    ("clf", clf3)
])

# Comitê de Votação
voting_classifier = VotingClassifier(estimators=[
    ("clf1", classifier1),
    ("clf2", classifier2),
    ("clf3", classifier3)
], voting="hard")

# Pipeline final
final_pipeline = Pipeline([
    ("voting_classifier", voting_classifier)
])